# Week 2: Deep Neural Networks for Time Series
## Preparing features and labels
we have to divide our data into features and labels. In this case our feature is effectively a number of values in the series, with our label being the next value.
We'll call that number of values that will treat as our feature, the **window size**, where we're taking a window of the data and training an ML model to predict the next value.
So for example, if we take our time series data, say, 30 days at a time, we'll use 30 values as the feature and the next value is the label. Then over time, we'll train a neural network to match the 30 features to the single label.
Create some data for us:

In [1]:
import tensorflow as tf
dataset = tf.data.Dataset.range(10) # Make a range of 10 values
for val in dataset:
    print(val.numpy())



0
1
2
3
4
5
6
7
8
9


In [33]:
dataset = tf.data.Dataset.range(10)
# expand dataset using windowing:
dataset = dataset.window(5, shift=1, drop_remainder=True) # size of the window and how much to shift each time
# drop_remainder - will truncate the data and will give us only windows of 5 items
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [34]:
# Create numpy lists:
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [35]:
# Split data into features and labels:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x,y in dataset:
    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [36]:
# Shuffle data before training:
dataset = dataset.shuffle(buffer_size=10) # 10 - is amount of data items that we have
# Batch some data:
dataset = dataset.batch(2).prefetch(1) # batch the data in the sets of two
for x,y in dataset:
    print("x = ", x.numpy())
    print("y = ", y.numpy())


x =  [[1 2 3 4]
 [5 6 7 8]]
y =  [[5]
 [9]]
x =  [[2 3 4 5]
 [0 1 2 3]]
y =  [[6]
 [4]]
x =  [[3 4 5 6]
 [4 5 6 7]]
y =  [[7]
 [8]]
